**FinLand** *The land use prediction app*
RGB data obtained from EuroSat : http://madm.dfki.de/downloads was  made into numpy array (Files : https://drive.google.com/open?id=10hrjspOW-3NaGzbehPyD1jZMZgHfbdLE) input into a  Transfer Learning model with a Fine Tuned VGG-16 architecture to obtain a high accuracy prediction model

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from keras import backend as k #
import time
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


# Any results you write to the current directory are saved as output.
import os
print(os.listdir("../input"))




['images.csv.npy', 'labels.csv.npy']


In [4]:

#loading base model
base_model = VGG16(weights='imagenet', include_top=False,input_shape = (64, 64,3))
base_model.trainable= False
base_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [5]:
#batch_size to train
batch_size = 32
# number of output classes
nb_classes = 10
# number of epochs to train
nb_epoch = 10

In [6]:
#since vgg16 uses 224x224x3 dimension images, we decided to tweakthe input layer a bit and freeze all the layers
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(base_model)
 
# Add new layers
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(Flatten())
model.add(layers.Dense(nb_classes, activation='softmax', name ='output'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2, 2, 512)         262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2, 512)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
output (Dense)               (None, 10)                20490     
Total params: 14,997,834
Trainable params: 283,146
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
loaded_images = np.load('../input/images.csv.npy')
loaded_labels = np.load('../input/labels.csv.npy')

In [8]:
loaded_images

array([[[[-0.4921875, -0.265625 , -0.296875 ],
         [-0.4921875, -0.265625 , -0.296875 ],
         [-0.4765625, -0.2890625, -0.2734375],
         ...,
         [-0.359375 , -0.296875 , -0.2109375],
         [-0.3828125, -0.328125 , -0.203125 ],
         [-0.3359375, -0.28125  , -0.15625  ]],

        [[-0.5      , -0.2734375, -0.2890625],
         [-0.4765625, -0.2734375, -0.28125  ],
         [-0.4765625, -0.2890625, -0.2890625],
         ...,
         [-0.359375 , -0.296875 , -0.2109375],
         [-0.3828125, -0.3203125, -0.21875  ],
         [-0.3359375, -0.2734375, -0.171875 ]],

        [[-0.46875  , -0.265625 , -0.2578125],
         [-0.46875  , -0.265625 , -0.2734375],
         [-0.46875  , -0.28125  , -0.28125  ],
         ...,
         [-0.2890625, -0.21875  , -0.1484375],
         [-0.2890625, -0.21875  , -0.1484375],
         [-0.3359375, -0.265625 , -0.1953125]],

        ...,

        [[-0.3828125, -0.3046875, -0.2109375],
         [-0.375    , -0.296875 , -0.203125 ]

In [9]:
loaded_images[1]

array([[[-0.3671875, -0.375    , -0.328125 ],
        [-0.3671875, -0.375    , -0.3125   ],
        [-0.4140625, -0.4140625, -0.3359375],
        ...,
        [-0.3046875, -0.3203125, -0.28125  ],
        [-0.359375 , -0.3828125, -0.328125 ],
        [-0.359375 , -0.3828125, -0.3125   ]],

       [[-0.3671875, -0.375    , -0.328125 ],
        [-0.3671875, -0.375    , -0.3125   ],
        [-0.40625  , -0.40625  , -0.328125 ],
        ...,
        [-0.3046875, -0.3203125, -0.28125  ],
        [-0.359375 , -0.3828125, -0.3125   ],
        [-0.359375 , -0.3828125, -0.3125   ]],

       [[-0.34375  , -0.3515625, -0.2890625],
        [-0.34375  , -0.3515625, -0.2890625],
        [-0.3984375, -0.3984375, -0.3203125],
        ...,
        [-0.2578125, -0.296875 , -0.25     ],
        [-0.34375  , -0.3671875, -0.296875 ],
        [-0.390625 , -0.4140625, -0.328125 ]],

       ...,

       [[-0.3359375, -0.3515625, -0.3125   ],
        [-0.3359375, -0.3515625, -0.3125   ],
        [-0.3203125, -

In [10]:
loaded_labels

array(['Residential', 'HerbaceousVegetation', 'River', ..., 'River',
       'PermanentCrop', 'Highway'], dtype='<U20')

In [11]:
loaded_labels.shape

(26408,)

In [12]:
for i in range(26408):
    if(loaded_labels[i] == 'AnnualCrop'):
        loaded_labels[i] = 0
    if(loaded_labels[i] == 'Forest'):
        loaded_labels[i] = 1
    if(loaded_labels[i] == 'HerbaceousVegetation'):
        loaded_labels[i] = 2
    if(loaded_labels[i] == 'Highway'):
        loaded_labels[i] = 3
    if(loaded_labels[i] == 'Industrial'):
        loaded_labels[i] = 4
    if(loaded_labels[i] == 'Pasture'):
        loaded_labels[i] = 5
    if(loaded_labels[i] == 'PermanentCrop'):
        loaded_labels[i] = 6
    if(loaded_labels[i] == 'Residential'):
        loaded_labels[i] = 7
    if(loaded_labels[i] == 'River'):
        loaded_labels[i] = 8
    if(loaded_labels[i] == 'SeaLake'):
        loaded_labels[i] = 9

In [13]:
loaded_labels = loaded_labels.astype(np.int)
loaded_labels

array([7, 2, 8, ..., 8, 6, 3])

In [14]:
X_train, X_test, y_train, y_test = train_test_split(loaded_images, loaded_labels, test_size=0.2, random_state=4)

In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

(21126, 64, 64, 3)
(21126,)
(5282, 64, 64, 3)
(5282,)
21126 train samples
5282 test samples


In [16]:
X_train

array([[[[ 0.1953125,  0.0859375,  0.0625   ],
         [ 0.21875  ,  0.078125 ,  0.0625   ],
         [ 0.25     ,  0.0625   ,  0.046875 ],
         ...,
         [ 0.546875 ,  0.2109375,  0.140625 ],
         [ 0.5859375,  0.25     ,  0.1953125],
         [ 0.5625   ,  0.2265625,  0.171875 ]],

        [[ 0.1953125,  0.0859375,  0.0625   ],
         [ 0.203125 ,  0.0859375,  0.0625   ],
         [ 0.2265625,  0.0625   ,  0.0390625],
         ...,
         [ 0.546875 ,  0.2109375,  0.140625 ],
         [ 0.5859375,  0.25     ,  0.1953125],
         [ 0.5625   ,  0.2265625,  0.171875 ]],

        [[ 0.1171875,  0.0078125, -0.015625 ],
         [ 0.125    ,  0.0078125, -0.015625 ],
         [ 0.078125 , -0.0625   , -0.09375  ],
         ...,
         [ 0.5625   ,  0.2265625,  0.15625  ],
         [ 0.59375  ,  0.2578125,  0.203125 ],
         [ 0.546875 ,  0.2109375,  0.15625  ]],

        ...,

        [[ 0.2734375,  0.078125 ,  0.0234375],
         [ 0.2421875,  0.0859375,  0.015625 ]

In [17]:
y_train[21125]

2

In [18]:
from keras.utils import np_utils

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)


In [19]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [20]:
from keras.preprocessing.image import ImageDataGenerator

# create generators  - training data will be augmented images
validationdatagenerator = ImageDataGenerator()
traindatagenerator = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=15,zoom_range=0.1 )

batchsize=8
train_generator=traindatagenerator.flow(X_train, y_train, batch_size=batchsize) 
validation_generator=validationdatagenerator.flow(X_test, y_test,batch_size=batchsize)

In [21]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [22]:
model.fit_generator(train_generator, steps_per_epoch=int(len(X_train)/batchsize), 
                    epochs=10, validation_data=validation_generator, 
                    validation_steps=int(len(X_test)/batchsize))


Epoch 1/10
2640/2640 [==============================] - 69s 26ms/step - loss: 0.5997 - acc: 0.7935 - val_loss: 0.4216 - val_acc: 0.8581
Epoch 2/10
2640/2640 [==============================] - 68s 26ms/step - loss: 0.4534 - acc: 0.8458 - val_loss: 0.3867 - val_acc: 0.8701
Epoch 3/10
2640/2640 [==============================] - 68s 26ms/step - loss: 0.4122 - acc: 0.8607 - val_loss: 0.3482 - val_acc: 0.8866
Epoch 4/10
2640/2640 [==============================] - 68s 26ms/step - loss: 0.3969 - acc: 0.8652 - val_loss: 0.3430 - val_acc: 0.8910
Epoch 5/10
2640/2640 [==============================] - 67s 26ms/step - loss: 0.3813 - acc: 0.8703 - val_loss: 0.4517 - val_acc: 0.8739
Epoch 6/10
2640/2640 [==============================] - 67s 26ms/step - loss: 0.3751 - acc: 0.8751 - val_loss: 0.3253 - val_acc: 0.8984
Epoch 7/10
2640/2640 [==============================] - 67s 26ms/step - loss: 0.3634 - acc: 0.8759 - val_loss: 0.3443 - val_acc: 0.8946
Epoch 8/10
2640/2640 [==========================

In [23]:
imagepredict = np.expand_dims(X_train[0], axis=0)
    #print(imagepredict.shape)
print(model.predict(imagepredict))
print(y_train[0])

[[3.69955933e-06 8.51920357e-12 4.73100499e-06 1.38105918e-03
  5.70686348e-02 1.55522366e-06 9.28164601e-01 1.33693805e-02
  6.31813100e-06 5.08355684e-12]]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [24]:
from keras.models import load_model
model.save('land_predict.h5')

In [ ]:
x=load_model('land_predict.h5')

In [ ]:
imagepredict = np.expand_dims(X_train[1], axis=0)
    #print(imagepredict.shape)
print(x.predict(imagepredict))
print(y_train[1])

In [ ]:
import keras
print(keras.__version__)
